In [ ]:
import numpy as np
import random
from scipy import signal
from IPython.display import Audio
import matplotlib.pyplot as plot
import math
import itertools
import functools
from fractions import gcd

In [ ]:
## Normalized and Real Planet data arrays

masses = [.0553, .815, 1, .107, 317.83, 95.16, 14.54, 17.15, .0022]
densities = [.985, .951, 1, .714, .214, .125, .230, .297, .336] ## all are less than Earth!
radii = [.384, .952, 1, .531, 10.517, 8.552, 3.929, 3.829, .187]
velocities = [.384, .926, 1, .450, 5.32, 3.172, 1.903, 2.10, .108]
periods = [58.785, 243.690, 1, 1.029, 0.415, 0.445, 0.720, 0.673, 6.405]
days = [175.942, 116.750, 1, 1.027, 0.414, 0.444, 0.718, 0.671, 6.387] ## why are some days so far from the period?

temperatures = [167, 464, 15, -63, -108, -139, -197, -201, -242]  ## in celcius
moons = [0, 0, 1, 2, 79, 82, 27, 14, 5]

orbital_periods = [88, 224.7, 365.2, 687, 4331, 10747, 30589, 59800, 90560]
orbital_hours = [period*24 for period in orbital_periods]

## masses --> frequency centroid
## diameter --> frequency range
## density --> range/layers
## days --> harmony change rate
## temperatures --> rhythms

real_masses = [item*5.9722 for item in masses]
real_densities = [item*5514 for item in densities]
real_diameters = [item*2*6356.752 for item in radii]
real_velocities = [item*11.2 for item in velocities]
real_periods = [item*23.9345 for item in periods]
real_days = [item*24 for item in days]

normal_masses = [item/max(masses) for item in masses]
normal_densities = [item/max(densities) for item in densities]
normal_diameters = [item/max(real_diameters) for item in real_diameters]
normal_velocities = [item/max(velocities) for item in velocities]
normal_periods = [item/max(periods) for item in periods]
normal_days = [item/max(days) for item in days]
normal_orbital_periods = [item/max(orbital_periods) for item in orbital_periods]

kelvin_temperatures = [item+273.15 for item in temperatures]
normal_temperatures = [item/max(kelvin_temperatures) for item in kelvin_temperatures]

days_to_pluto = []
for i in range(len(real_days)):
    days_to_pluto.append(orbital_periods[8]/real_days[i])
print(days_to_pluto)

#print(normal_densities)

centroids = np.geomspace(70, 17000, num=19, endpoint=True)
#time = np.linspace(0,1,23)
#centroids = np.cos(2*np.pi*.25*time)
#plot.plot(np.geomspace(70, 6000, num=18, endpoint=True))

probs = np.ones(19)
for i in range(len(probs)):
    if i != 0:
        probs[i] = (probs[i-1])/2

planet_centroids = []
for mass in normal_masses:
    for j in range(len(probs)):
        if mass >= probs[j]:
            planet_centroids.append(centroids[j])
            probs = np.delete(probs,j)
            centroids = np.delete(centroids,j)
            break

log_freq_bins = np.concatenate((np.linspace(20,100,9), np.linspace(200,1000,9), np.linspace(2000,10000,9)))
octave_bands_440 = [(2**j)*20 for j in range(0,11)]

planet_freq_ranges = []
planet_centroid_bins = []

##these are lower cutoff frequencies; bins are in between two values        
bark_scale = [0, 20, 100, 200, 300, 400, 510, 630, 770, 920, 1080, 1270, 1480, 1720, 2000, 2320, 2700, 3150, 3700, 4400, 5300, 6400, 7700, 9500, 12000, 15500, 17000, 20000]

for centroid_index in range(len(planet_centroids)):
    for i in range(len(bark_scale)):
        if planet_centroids[centroid_index] <= bark_scale[i]:
            mass = normal_diameters[centroid_index]
            mass = round(mass)
            planet_centroid_bins.append(bark_scale[i-1])
            planet_freq_ranges.append((bark_scale[i-1],bark_scale[i+mass]))
            break

planet_bin_cutoffs = octave_bands_440
planet_bin_cutoffs.remove(40)

planet_bins = [(planet_bin_cutoffs[i], planet_bin_cutoffs[i+1]) for i in range(len(planet_bin_cutoffs)-1)]
planet_bins[1] = (40, 160)

##planet bins are currently in order of mass --> jupiter is first, pluto is last

In [ ]:
##This is the crux of the harmonic model for each planet. There are 16 elements across the nine atmospheres
## and these elements will split every third octave into 16 justly tempered parts. Each planet will then be
## assigned a certain number of third octaves according to their mass and diameter.

## Elemental frequencies!
elements = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Na', 'Mg', 'Al', 'S',
           'Ar', 'K', 'Ca', 'Fe', 'Kr', 'Xe']

ratios_et = [1/1, 17/16, 9/8, 19/16, 5/4, 21/16, 11/8, 23/16, 3/2,
            25/16, 13/8, 27/16, 7/4, 29/16, 15/8, 31/16, 2/1]

## we took out the perfect fifth -- now this represents splitting every third octave into 16 frequencies
ratios_ji = [1/1, 5/4, 6/5, 7/6, 9/8, 11/10, 13/12, 15/14, 17/16, 
             19/18, 21/20, 23/22, 25/24, 27/26, 29/28, 31/30]

ratios_ji.sort(reverse=False)

elemental_ratio = {element: ratio for (element, ratio) in zip(elements, ratios_ji)}

##just intonation is a bit harder to calculate. right now, ratios_ji
##have interval values only up to the perfect fifth. The next step
##is inverting the first half of the values so that they are
##between the fifth and the octave, but I don't have time for that so 
## I'll just use ET for now

##octaves of C -- index is associated with octave number (index = 4 --> C4 (middle C))
octaves_C = [16.35, 32.70, 65.41, 130.81, 261.63, 523.25, 1046.50, 2093, 4186.01, 8372.02, 16744.04]

##third octaves, justly intonated and adjusted at each octave of C
third_octaves_ji = []
C_num = 0
curr_freq = octaves_C[0]

for i in range(len(octaves_C)*3):
    if i%3 == 0:
        third_octaves_ji.append(octaves_C[C_num])
        curr_freq = octaves_C[C_num]
        C_num += 1
    else:
        third_octaves_ji.append(curr_freq)
    curr_freq = curr_freq*1.25

##this is with JI third -- we lose some of the octave this way
planet_thirds_ji = [item for item in third_octaves_ji if item >= octaves_C[4]]

third_octaves_et = []
et_ratio = 2**(1/12)
for item in octaves_C:
    third_octaves_et.append(item)
    third_octaves_et.append(item*(et_ratio**4))
    third_octaves_et.append(item*(et_ratio**8))

## this is with ET thirds across the board
planet_thirds_et = [item for item in third_octaves_et if ((item >= octaves_C[4]) & (item <= octaves_C[-1]*et_ratio**4))]

In [ ]:
## MASTER VALUES FOR RELATIVE VALUES
REAL_HRS_PER_MUSICAL_SECOND = 4520 #hours per musical second -- derived from time it takes for pluto to orbit
TEMPERATURE = 10 #division of day into rhythms -- will be used to calculate speed of pulse
ELEMENTS = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Na', 'Mg', 'Al', 'S',
           'Ar', 'K', 'Ca', 'Fe', 'Kr', 'Xe']
##DURATIONS = [8,7,6,5,4,3,2,1,1,2,3,4,5,6,7,8] this was used to represent a speeding up as temperatures get hotter
## throughout each day -- can make one of these for each planet maybe...? Still don't know how I'm going to use
## temperature
DURATIONS = [9,7,9,6,5,4,1,1,1,1,4,1,1,1,2,2] ##based on frequency of occurance
DURATIONS_SUM = sum(DURATIONS)
DENSEST_CHORD = 7
TOTAL_TIME = 480 #seconds (8 min)
ELEMENTAL_DURATIONS = {element: duration for (element, duration) in zip(ELEMENTS, DURATIONS)}
ELEMENTAL_RATIOS = {element: ratio for (element, ratio) in zip(ELEMENTS, ratios_ji)}
FS = 48000
PLANET_THIRDS_ET = [item for item in third_octaves_et if ((item >= octaves_C[4]) & (item <= octaves_C[-1]*et_ratio**4))]
TOTAL_REAL_HRS_TRAVERSED = REAL_HRS_PER_MUSICAL_SECOND*TOTAL_TIME
REALITY_FACTOR = 4 ## to make it possible to compute all planets
DAY = (4222.608/4520)*REALITY_FACTOR ##length of mercury day, in hrs, divided by total hrs per second; yields normaled value of day in seconds

## create the distance matrix given a set of pitch sets
def build_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        for j in range(len(pitch_sets)):
            row.append(len(pitch_sets[i].intersection(pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

## build pitch sets (no repeating elements, in sorted order) from list of elements
def build_pitch_sets(element_list, size):
    return list(map(set, itertools.combinations(element_list, size)))

##get a rhythmic firmus for each day -- each element will be paired with a duration in a tuple
def get_rhythmic_firmus(element_list):
    return list(random.sample(element_list, k=len(element_list)))

##randomly get elements right now -- should substitute markov model here
## will need to input a distance matrix
def get_day_elements(element_list):
    return random.sample(element_list, k=merc_chord_size)

## if element at each step of a rhythmic firmus is in the pitch set, create a sine wave for it
## otherwise, append an array of zeros (i.e. a rest)
## right now: casting zeros array to int -- might cause issues of a few samples down the road
def make_day_from_pitch_and_rhythm(pitch_set, rhythmic_firmus, octaves, dur_factor):
    day = []
    for element in rhythmic_firmus:
        duration = ELEMENTAL_DURATIONS[element]*dur_factor
        if element in pitch_set:
            octave = random.sample(octaves, k=1)
            freq = (ELEMENTAL_RATIOS[element])*octave[0]
            time = np.arange(0, duration, 1/FS)
            note = np.sin(2*np.pi*freq*time)
            day.append(note)
        else:
            day.append(np.zeros(int(np.ceil(duration*FS))))
    return np.concatenate(day)

## random walk creation
def create_drunkards_walking_music(pitch_sets, distance_matrix, iterations):
    current_set = random.randint(0,len(pitch_sets)-1)
    drunkards_walking_music = [pitch_sets[current_set]]
    for i in range(iterations):
        r = random.uniform(0,1)
        j = 0
        while distance_matrix[current_set][j] < r:
            j +=1
        drunkards_walking_music.append(list(pitch_sets[j]))
        current_set = j
    return(drunkards_walking_music)

##create random envelope for a given array length and range values
def random_adsr(array_length, a_range, d_range, s_range, r_range, fs):
    length = array_length

    attack_max_endpoint = length*a_range
    decay_max_endpoint = length*d_range
    sustain_max_endpoint = length*s_range
    release_max_endpoint = length*r_range

    attack_end = int(np.ceil(random.uniform(0,1)*attack_max_endpoint))
    decay_end = int(np.ceil(random.uniform(0,1)*decay_max_endpoint))
    sustain_end = int(np.ceil(random.uniform(0,1)*sustain_max_endpoint))
    release_end = length - (attack_end + decay_end + sustain_end)

    attack = np.geomspace(0.001, random.uniform(.8,1), num = attack_end, endpoint = True)
    decay = np.geomspace(attack[-1], random.uniform(.4,.8), num = decay_end, endpoint = False)

    time = np.arange(0, 1/fs*sustain_end, 1/fs)
    sustain = random.uniform(.05,.1)*np.sin(2*np.pi*random.randint(17000,20000)*time) + decay[-1]
    
    release = np.geomspace(sustain[-1], 0.001, num = release_end, endpoint = True)
    
    to_return = np.concatenate((attack, decay, sustain, release))
    
    return np.concatenate((attack, decay, sustain, release))[0:length]

def make_planet(planet):
    day = DAY*planet['relative_day']
    counter = int(TOTAL_REAL_HRS_TRAVERSED/planet['day']/REALITY_FACTOR)
    print(counter)
    dur_factor = day/DURATIONS_SUM
    print(dur_factor*DURATIONS_SUM)
    chord_size = round(DENSEST_CHORD*planet['density'])
    print(chord_size)

    sets = build_pitch_sets(planet['atmosphere'], chord_size)
    matrix = build_distance_matrix(sets)
    drunkards = create_drunkards_walking_music(sets, matrix, counter)

    PLANET = []
    print(len(drunkards))
    for element_list in drunkards:
        rhythmic_firmus = get_rhythmic_firmus(ELEMENTS)
        day = make_day_from_pitch_and_rhythm(element_list, rhythmic_firmus, planet['octaves'], dur_factor)
        #envelope = random_adsr(len(day), .1, .05, .8, .05, FS)
        day = day#*envelope
        for i in range(chord_size-1):
            rhythmic_firmus = get_rhythmic_firmus(ELEMENTS)
            one_line = make_day_from_pitch_and_rhythm(element_list, rhythmic_firmus, planet['octaves'], dur_factor)
            np.add(day, one_line)
        PLANET.append(day)
    
    return np.concatenate(PLANET)
print(PLANET_THIRDS_ET)

In [ ]:
## Mercury 
## measurements proportional to Earth's
## mass -- 0.0553
## density -- 0.985
## escape velocity -- 0.384
## rotational period -- 58.785
## length of day -- 175.942

## not proportional to Earth
## mean temperature -- 167 C
## # of moons -- 0

mercury = {'mass': normal_masses[0], 
           'density': normal_densities[0],
           'velocity': normal_velocities[0],
           'period': normal_periods[0],
           'relative_day': normal_days[0],
           'day': real_days[0],
           'temperature': normal_temperatures[0],
           'moons': moons[0],
           'atmosphere': ['H', 'K', 'Mg', 'Na', 'O', 'Ca', 'Fe', 'Al', 'C', 'N', 'Xe', 'Kr', 'Ne', 'He'],
           'octaves': PLANET_THIRDS_ET[16:17]}

merc_day = DAY*mercury['relative_day']
merc_counter = int(TOTAL_REAL_HRS_TRAVERSED/mercury['day'])
## was used before making elemental rhythms based on occurance
## merc_durations  = [duration*merc_day/DURATIONS_SUM for duration in DURATIONS]
merc_dur_factor = merc_day/DURATIONS_SUM
merc_chord_size = math.floor(DENSEST_CHORD*mercury['density'])

merc_sets = build_pitch_sets(mercury['atmosphere'], merc_chord_size)
merc_matrix = build_distance_matrix(merc_sets)
merc_drunkards = create_drunkards_walking_music(merc_sets, merc_matrix, merc_counter)

MERCURY = []
for element_list in merc_drunkards:
    rhythmic_firmus = get_rhythmic_firmus(ELEMENTS)
    day = make_day_from_pitch_and_rhythm(element_list, rhythmic_firmus, mercury['octaves'], merc_dur_factor)
    envelope = random_adsr(len(day), .001, .001, .997, .001, FS)
    day = day*envelope
    for i in range(merc_chord_size-1):
        rhythmic_firmus = get_rhythmic_firmus(ELEMENTS)
        one_line = make_day_from_pitch_and_rhythm(element_list, rhythmic_firmus, mercury['octaves'], merc_dur_factor)
        envelope = random_adsr(len(day), .05, .05, .89, .01, FS)
        one_line = one_line*envelope
        np.add(day, one_line)
    MERCURY.append(day)
    
Audio(np.concatenate(MERCURY),rate=FS)

In [ ]:
## Venus 
## measurements proportional to Earth's
## mass -- 0.815
## density -- 0.951
## escape velocity -- 0.926
## rotational period -- 243.690
## length of day -- 116.750

## not proportional to Earth
## mean temperature -- 464 C
## # of moons -- 0

venus = {'mass': normal_masses[1],
           'density': normal_densities[1], 
           'velocity': normal_velocities[1], 
           'period': normal_periods[1], 
           'relative_day': normal_days[1],
           'day': real_days[1],
           'temperature': normal_temperatures[1],
           'moons': moons[1],
           'atmosphere': ['C', 'O', 'N', 'S', 'Ar', 'H', 'He', 'Ne'],
           'octaves': planet_thirds_et[14:15]}

VENUS = make_planet(venus)
Audio(VENUS, rate=FS)

In [ ]:
## Earth
## rote Earth measurements
## mass -- 5.9722 kg x 10^24
## density -- 5514 kg/m^3
## escape velocity -- 11.2 km/s
## rotational period -- 23.9345 hrs
## length of day -- 24 hrs 

## mean temperature -- 15 C
## # of moons -- 1

## temperature changes throughout the day
## could focus on one day and have the harmony change according to the hours
## no that's stupid -- do what is going to get this done faster, at least for now; there's limited time

earth = {'mass': normal_masses[2], 
           'density': normal_densities[2], 
           'velocity': normal_velocities[2], 
           'period': normal_periods[2], 
           'relative_day': normal_days[2],
           'day': real_days[2], 
           'temperature': normal_temperatures[2],
           'moons': moons[2],
           'atmosphere': ['N', 'C', 'O', 'Ar', 'Ne', 'He', 'Kr', 'H'],
           'octaves': planet_thirds_et[13:14]}

EARTH = make_planet(earth)
Audio(EARTH, rate=FS)

In [ ]:
## Mars
## measurements proportional to Earth's
## mass -- 0.107
## density -- 0.714
## escape velocity -- 0.450
## rotational period -- 1.029
## length of day -- 1.027

## not proportional to Earth
## mean temperature -- -63 C ***
## # of moons -- 2

mars = {'mass': normal_masses[3], 
           'density': normal_densities[3], 
           'velocity': normal_velocities[3], 
           'period': normal_periods[3], 
           'relative_day': normal_days[3],
           'day': real_days[3],
           'temperature': normal_temperatures[3],
           'moons': moons[3],
           'atmosphere': ['C', 'O', 'N', 'Ar', 'H', 'Ne', 'Kr', 'Xe'],
           'octaves': planet_thirds_et[15:16]}

MARS = make_planet(mars)
Audio(MARS, rate=FS)

In [ ]:
## Jupiter
## measurements proportional to Earth's
## mass -- 317.83
## density -- 0.241
## escape velocity -- 5.32
## rotational period -- 0.415
## length of day -- 0.414

## not proportional to Earth
## mean temperature -- -108 C
## # of moons -- 79

jupiter = {'mass': normal_masses[4], 
           'density': normal_densities[4], 
           'velocity': normal_velocities[4], 
           'period': normal_periods[4], 
           'relative_day': normal_days[4],
           'day': real_days[4],
           'temperature': normal_temperatures[4],
           'moons': moons[4],
           'atmosphere': ['H', 'He', 'C', 'N', 'O'],
           'octaves': planet_thirds_et[0:5]}

JUPITER = make_planet(jupiter)
Audio(JUPITER, rate=FS)

In [ ]:
## Saturn
## measurements proportional to Earth's
## mass -- 95.16
## density -- 0.125
## escape velocity -- 3.172
## rotational period -- 0.445
## length of day -- 0.444

## not proportional to Earth
## mean temperature -- -139 C
## # of moons -- 82

saturn = {'mass': normal_masses[5], 
           'density': normal_densities[5], 
           'velocity': normal_velocities[5], 
           'period': normal_periods[5], 
           'relative_day': normal_days[5],
           'day': real_days[5],
           'temperature': normal_temperatures[5],
           'moons': moons[5],
           'atmosphere': ['H', 'He', 'C', 'N'],
           'octaves': planet_thirds_et[5:9]}

SATURN = make_planet(saturn)
Audio(SATURN, rate=FS)

In [ ]:
## Uranus
## measurements proportional to Earth's
## mass -- 14.54
## density -- 0.230
## escape velocity -- 1.903
## rotational period -- 0.720
## length of day -- 0.718

## not proportional to Earth
## mean temperature -- -197 C
## # of moons -- 27

uranus = {'mass': normal_masses[6], 
           'density': normal_densities[6], 
           'velocity': normal_velocities[6], 
           'period': normal_periods[6], 
           'relative_day': normal_days[6],
           'day': real_days[6], 
           'temperature': normal_temperatures[6],
           'moons': moons[6],
           'atmosphere': ['H', 'He', 'C'],
           'octaves': planet_thirds_et[11:13]}

URANUS = make_planet(uranus)
Audio(URANUS, rate=FS)

In [ ]:
## Neptune
## measurements proportional to Earth's
## mass -- 17.15
## density -- 0.297
## escape velocity -- 2.10
## rotational period -- 0.673
## length of day -- 0.671

## not proportional to Earth
## mean temperature -- -201 C
## # of moons -- 14

neptune = {'mass': normal_masses[7], 
           'density': normal_densities[7], 
           'velocity': normal_velocities[7], 
           'period': normal_periods[7], 
           'relative_day': normal_days[7],
           'day': real_days[7],
           'temperature': normal_temperatures[7],
           'moons': moons[7],
           'atmosphere': ['H', 'He', 'C'],
           'octaves': planet_thirds_et[9:11]}

NEPTUNE = make_planet(neptune)
Audio(NEPTUNE, rate=FS)

In [ ]:
## Pluto
## measurements proportional to Earth's
## mass -- 0.0022
## density -- 0.336
## escape velocity -- 0.108
## rotational period -- 6.405
## length of day -- 6.387

## not proportional to Earth
## mean temperature -- -247 - -233 C
## # of moons -- 5 

pluto = {'mass': normal_masses[8], 
           'density': normal_densities[8], 
           'velocity': normal_velocities[8], 
           'period': normal_periods[8], 
           'relative_day': normal_days[8],
           'day': real_days[8],
           'temperature': normal_temperatures[8],
           'moons': moons[8],
           'atmosphere': ['H', 'N', 'C', 'O'],
           'octaves': planet_thirds_et[17:18]}

PLUTO = make_planet(pluto)
Audio(PLUTO, rate=FS)